# Imports

In [ ]:
import os
import sys
import pandas as pd
from scipy.stats import norm, chi2
import statsmodels.api as sm
import numpy as np
from functools import partial
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

current_dir = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_dir, os.pardir))
grandparent_dir = os.path.abspath(os.path.join(parent_dir, os.pardir))
sys.path.insert(0, parent_dir)
sys.path.insert(0, grandparent_dir)
import cmds.portfolio_management_helper as pmh

plt.style.use("seaborn-v0_8-whitegrid")
PLOT_WIDTH, PLOT_HEIGHT = 8, 5
COLORS = ["blue", "red", "orange"]

warnings.filterwarnings('ignore')
pd.options.display.float_format = "{:.4f}".format
p = plt.rcParams

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
DATA_PATH = parent_dir + '/data/'
FILE_NAME = DATA_PATH + 'momentum_data.xlsx'
excess_ff_factors = pmh.read_excel_default(FILE_NAME, 
                                 sheet_name='factors (excess returns)',
                                 index_col='Date', parse_dates=True)

# Summary Tables

## Single Timeframe

In [ ]:

pmh.calc_summary_statistics(factors, annual_factor=12, correlations=False, provided_excess_returns=True,
                            keep_columns=['Annualized'], drop_columns=['VaR']).T.style.format("{:.2%}")

## Multiple Timeframes (with Correlations!)

In [ ]:
summary_table = pmh.calc_summary_statistics(ff_factors, annual_factor=12, provided_excess_returns=True, 
                            timeframes={'1927-2024':['1927', '2024'],
                                        '1927-1993':['1927', '1993'],
                                        '1994-2008':['1994', '2008'],
                                        '2009-2024':['2009', '2024']},
                            correlations=['MKT', 'HML'],
                            keep_columns=['Annualized Mean', 'Annualized Vol', 'Annualized Sharpe', 'Skewness', 'Correlation'])
summary_table.loc[summary_table.index.str.contains('UMD')]

# Mean-Variance Optimization

## Tangency Portfolio

In [ ]:
# Classic Tangency Weights 
pmh.calc_tangency_weights(rets, annual_factor=ANNUAL_FACTOR)

# Tangency Weights with a target Return
pmh.calc_tangency_weights(rets, target_ret_rescale_weights=0.0025, annual_factor=ANNUAL_FACTOR)

# Tangency Weights with a regularized Covariance Matrix
pmh.calc_tangency_weights(rets, cov_mat=COV_MAT)

# Tangency Portfolio with different return series for expected returns and covariance matrix
training_tangency_weights_expected_returns = pmh.calc_tangency_weights(
    training_stocks_excess_returns, # Series used to calculate the covriance matrix
    cov_mat=COV_MATRIX,
    expected_returns=stocks_expected_excess_return, # Series used to calculate the expected returns
    name="AQR Expected Returns"
)
training_tangency_weights_expected_returns.iloc[:3]

## GMV Portfolio

In [ ]:
# Classic GMV weights
pmh.calc_gmv_weights(rets, name='MV')

# GMV weights with a target Return
pmh.calc_gmv_weights(rets, target_ret_rescale_weights=0.0025, name='MV')

## Equal Weights Portfolio

In [ ]:
pmh.calc_equal_weights(rets)

## Combining Portfolios and Generating Summary Statistics

In [ ]:
IS_RETS = rets.loc[:'2021'].drop(columns=['BTC'])
OOS_RETS = rets.loc['2022':'2023'].drop(columns=['BTC'])

no_btc_wts = pmh.calc_tangency_weights(IS_RETS, annual_factor=ANNUAL_FACTOR,target_ret_rescale_weights=0.0025)
equal_wts = pmh.calc_equal_weights(IS_RETS)

target_portfolio = pmh.create_portfolio(OOS_RETS, weights=no_btc_wts.iloc[:, 0].to_dict(), port_name='Target')
equal_portfolio = pmh.create_portfolio(OOS_RETS, weights=equal_wts.iloc[:, 0].to_dict(), port_name='Equal')

pmh.calc_summary_statistics(pd.concat([target_portfolio, equal_portfolio], axis=1), annual_factor=ANNUAL_FACTOR, 
                            provided_excess_returns=True, keep_columns=['Annualized'], drop_columns='VaR').T.style.format("{:.2%}")

# Linear Factor Models

## Time Series Regression

In [ ]:
# For any LFPM task, need to account for IS and OOS periods when setting up the regression and testing performance
LAST_IN_SAMPLE_YEAR = "2018"
FIRST_OUT_OF_SAMPLE_YEAR = f"{(int(LAST_IN_SAMPLE_YEAR) + 1):.0f}"
COV_MATRIX = .5

excess_returns_in_sample = excess_returns.loc[:LAST_IN_SAMPLE_YEAR]
excess_returns_out_of_sample = excess_returns.loc[FIRST_OUT_OF_SAMPLE_YEAR:]

### Single Dependent Variable

In [ ]:
# Single dependent variable regressions are ususally used for LFD and Hedging tasks
# Return a summary table
pmh.calc_regression(excess_returns_in_sample, factors, include_intercept=True, annual_factor=ANNUAL_FACTOR)

# Return the model for use in prediction
model = pmh.calc_regression(excess_returns_in_sample, factors, include_intercept=True, annual_factor=ANNUAL_FACTOR, return_model=True)
model.predict(sm.add_constant(excess_returns_out_of_sample))    # Critical that you add the constant IF INTERCEPT IS INCLUDED
model.params[1:]    # Beta coefficients for hedging and replication problems

### Iterative Regression (Multiple Dependent Variables)

In [ ]:
# This function is typically used for the time-series regression step in LFPM tasks
capm_ts = pmh.calc_iterative_regression(rets.loc['1981':], factors.loc['1981':],
                              warnings=False,
                              keep_columns=['Alpha', 'Beta', 'R-Squared',
                                            'Annualized'])
display(capm_ts)

### Mean Absolute Error Test

**Note:** This MAE test is different from the CS MAE test because we measure the MAE of the *alphas* across the TS regressions.

In [ ]:
pd.DataFrame((capm_ts['Annualized Alpha']).abs().mean(), columns = ['MAE (%)'], index = ['CAPM'])

In [ ]:
# Fama-French 5-Factor Test
FF5F = ['MKT', 'SMB', 'HML', 'RMW', 'CMA']
ff5f_ts_test = pmh.calc_iterative_regression(portfolios, factors[FF5F], annual_factor=12,intercept=True, 
                                            keep_columns=['Annualized Alpha', 'R-Squared'])
display(ff5f_ts_test)
print(f'Mean-Absolute-Error: {ff5f_ts_test['Annualized Alpha'].abs().sum() / len(ff5f_ts_test):.2%}\
      \nMin-Absolute-Error: {ff5f_ts_test['Annualized Alpha'].abs().idxmin()} - {ff5f_ts_test['Annualized Alpha'].abs().min():.2%}\
      \nMax-Absolute-Error: {ff5f_ts_test['Annualized Alpha'].abs().idxmax()} - {ff5f_ts_test['Annualized Alpha'].abs().max():.2%}')

## Time Series Regression with Time-Varying Betas

In [ ]:
# This function is used to assess the accuracy of a replication strategy while the model is re-trained through time
def OOS_strat(df, factors, start, window_size=None, intercept=True):
    y = df
    if intercept:
        X = sm.add_constant(factors)
    else:
        X = factors

    forecast_err, null_err,oos_predictions,null_predictions = [], [],[],[]

    for i,j in enumerate(df.index):
        if i >= start:
            if window_size:
                begin = i - window_size
            else:
                begin = 0
            currX = X.iloc[begin:i]
            currY = y.iloc[begin:i]
            reg = sm.OLS(currY, currX, missing = 'drop').fit()
            null_forecast = currY.mean()
            reg_predict = reg.predict(X.iloc[[i]])
            actual = y.iloc[[i]]
            oos_predictions.append(reg_predict.T)
            null_predictions.append(pd.DataFrame([[reg_predict.index[0]]], columns = ['date'], index = [null_forecast]))
            forecast_err.append(reg_predict.values - actual)
            null_err.append(null_forecast.values - actual)
            
    RSS = (np.array(forecast_err)**2).sum()
    TSS = (np.array(null_err)**2).sum()
    predictions_df = pd.concat(oos_predictions).T.drop_duplicates()
    null_predictions_df = pd.concat(null_predictions).T
    
    return ((1 - RSS/TSS),reg,predictions_df,null_predictions_df)

# Example of how to use the function
OOS_r2, OOS_model, OOS_forecasts, null_predictions_df = OOS_strat(USO,features, features.loc[:'2017'].shape[0])
OOS_pred = OOS_forecasts.to_frame('OOS Forecast')
OOS_pred

print(f'Out-of-sample R-squared: {OOS_r2:.2%}')

corr = pmh.calc_correlations(pd.concat([OOS_pred, USO.loc['2018':'2023']], axis=1), 
                             return_heatmap=False, print_highest_lowest=False).iloc[0, 1]
print(f'Correlation between the out-of-sample forecast and USO: {corr:.2%}')

## Cross-Sectional Regression

### Mean Absolute Error Test

**Note:** This MAE test is different from the TS MAE test because we measure MAE in the cross-sectional regression as the sum of *error residuals*

In [ ]:
capm_cs_test = pmh.calc_cross_section_regression(portfolios, factors['MKT'].to_frame(),provided_excess_returns=True, annual_factor=12, 
                                                 name='CAPM',keep_columns=['R-Squared', 'Annualized Eta', 'Annualized Lambda', 
                                                                           'TS Annualized MAE', 'CS Annualized MAE']).T
aqr_cs_test = pmh.calc_cross_section_regression(portfolios, factors[AQR],provided_excess_returns=True, annual_factor=12, 
                                                 name='AQR',keep_columns=['R-Squared', 'Annualized Eta', 'Annualized Lambda', 
                                                                           'TS Annualized MAE', 'CS Annualized MAE']).T
ff3f_cs_test = pmh.calc_cross_section_regression(portfolios, factors[FF3F],provided_excess_returns=True, annual_factor=12, 
                                                 name='FF3F',keep_columns=['R-Squared', 'Annualized Eta', 'Annualized Lambda', 
                                                                           'TS Annualized MAE', 'CS Annualized MAE']).T
ff5f_cs_test = pmh.calc_cross_section_regression(portfolios, factors[FF5F],provided_excess_returns=True, annual_factor=12, 
                                                 name='FF5F',keep_columns=['R-Squared', 'Annualized Eta', 'Annualized Lambda', 
                                                                           'TS Annualized MAE', 'CS Annualized MAE']).T
pd.concat([capm_cs_test, aqr_cs_test, ff3f_cs_test, ff5f_cs_test], axis=1)

### Bivariate Significance Test

In [ ]:
# Calculating Residuals Covariance Matrix
resid = pd.DataFrame()
for pf in pf_excess_rets.columns:
    r = pmh.calc_regression(pf_excess_rets.loc['1981':, pf], factors.loc['1981':, 'Mkt-RF'].to_frame('Mkt-RF'), 
                            annual_factor=12, warnings=False, return_model=True, return_fitted_values=False)
    r = r.resid.to_frame(pf)
    resid = pd.concat([resid, r], axis=1)

# Conducting the H- and t-tests (requires residuals from above and alphas from iterative regression)
T = pf_excess_rets['1981':].shape[0]
SR = (factors['1981':]['Mkt-RF'].mean() / factors['1981':]['Mkt-RF'].std()) #* np.sqrt(12)
Sigma = resid.cov()
Sigma_inv = pd.DataFrame(np.linalg.inv(Sigma), index=Sigma.index, columns=Sigma.columns)
alpha = capm_ts['Alpha']    # Not Annualized

H = T * (1 + SR**2)**(-1) * (alpha @ Sigma_inv @ alpha)

print('H = {:.2f}'.format(H))
pvalue = 1 - chi2.cdf(H, df=25)
print('p-value = {:.4f}'.format(pvalue))

# Time Diversification

## Probability of Under Performance

In [ ]:
def prob_under(mu, sigma, c, h):
    return norm.cdf(((c-mu)/sigma) * np.sqrt(h))

mu = barn_rets.loc['1965':'1999', 'LOG_SPX-XS'].mean()
sigma = barn_rets.loc['1965':'1999', 'LOG_SPX-XS'].std()

# NOTE: We are using 0 for the c parameter here because we are using excess returns
print(f'From 1965-1999, Monthly:\n\tPr(SPX Returns < RF Returns) = {prob_under(mu, sigma, c=0, h=1):.2%}')

In [ ]:
x = np.arange(0, 12*30)
y = prob_under(mu, sigma, c=0, h=x)

fig, ax = plt.subplots()
ax.plot(x, y)
ax.set_title('Change in Probability of Under-Performance\nover Different Time Horizons\n(1965-1999)')
ax.set(xlabel='Holding Period\n(Months)', ylabel='Pr(SPX < RF)')

In [ ]:
# This example is assessing probability of outperformance. Trick: Subtract the probability from 1
mu = aqr_log_rets.loc['2009':, 'UMD'].mean()
sigma = aqr_log_rets.loc['2009':, 'UMD'].std()
c = aqr_log_rets.loc[:, 'MKT'].mean()

print(f'Single Period:\n\tPr(UMD Mean Excess Rets > MKT) = {1-prob_under(mu, sigma, c=c, h=1):.2%}')
print(f'15-Year:\n\tPr(UMD Mean Excess Rets > MKT) = {1-prob_under(mu, sigma, c=c, h=12*15):.2%}')

## Analyzing Time Diversification on Returns

In [ ]:
pmh.calc_summary_statistics(aqr_log_rets, annual_factor=12*15, # This annual factor scales the log returns for time diversification analysis
                            provided_excess_returns=True,
                            keep_columns=['Annualized Mean', 'Annualized Vol', 'Annualized Sharpe']).T.style.format('{:.2%}')

# Forecasting

In [ ]:
# NOTE: The features parameter in the below is shifted down to make this forecasting regression
model = pmh.calc_regression(target, features, annual_factor=ANNUAL_FACTOR, return_model=True)
pmh.calc_regression(target, features, annual_factor=ANNUAL_FACTOR,
                    keep_columns=['R-Squared', 'Alpha', 'Beta'], 
                    drop_columns=['Annualized']).T

In [ ]:
wts = 0.5 + 50 * model.predict(sm.add_constant(features))
wts = pd.DataFrame(wts, wts.index, ['Weights'])
rets = pd.DataFrame(target.iloc[1:].values * wts.values, wts.index, ['Strat Returns'])
rets.style.format("{:.2%}")

In [ ]:
# Rolling OOS Forecasting Performance
OOS_r2, OOS_reg_params, OOS_forecasts, null_predictions_df = OOS_strat(USO,features, features.loc[:'2017'].shape[0])
OOS_pred = OOS_forecasts.to_frame('OOS Forecast')
OOS_pred

In [ ]:
corr = pmh.calc_correlations(pd.concat([OOS_pred, USO.loc['2018':'2023']], axis=1), 
                             return_heatmap=False, print_highest_lowest=False).iloc[0, 1]
print(f'Correlation between the out-of-sample forecast and USO: {corr:.2%}')

# Carry Trade

## Static

In [ ]:
spot_returns = log_ex_rates - log_ex_rates.shift(1)
rate_returns = log_rf_rates.drop('USD', axis=1) - log_rf_rates[['USD']].values
xs_log_rets = spot_returns + rate_returns

(pmh.calc_summary_statistics(xs_log_rets, annual_factor=252, provided_excess_returns=True,
                            keep_columns=['Annualized Mean', 'Annualized Vol', 'Annualized Sharpe',])
                            .T.style.format("{:.2%}"))

In [ ]:
# Analyzing the Carry Trade Performance
mean = xs_log_ret.mean().values[0] * ANNUAL_FACTOR
spread = rate_parity.mean() * ANNUAL_FACTOR
curr_diff = spot_diff.sum().values[0]
pd.DataFrame(
            [mean, -spread, curr_diff], ['Mean Excess Log Returns', 'Mean Rate Spread', 'Spot Difference'], ['Values']
             ).style.format("{:.2%}")

## Dynamic - Using Regression to Estimate Expected Returns

In [ ]:
factor = -rate_returns.shift(1)
regressions = {}
for currency in spot_returns.columns:
    regressions[currency] = pmh.calc_regression(spot_returns[[currency]], factor[[currency]], annual_factor=252, 
                                                intercept=True, return_model=True, warnings=False)
    
pd.DataFrame(columns=xs_log_rets.columns, index=['Annualized Alpha', 'Beta', 'R-Squared'], 
             data=[[regressions[currency].params[0] * 252 for currency in spot_returns.columns],
                   [regressions[currency].params[1] for currency in spot_returns.columns],
                   [regressions[currency].rsquared for currency in spot_returns.columns]])

In [ ]:
# (1)
pred_xs_log_rets = pd.DataFrame()
for currency in spot_returns.columns:
    pred_xs_log_rets[currency] = regressions[currency].params[0] + (regressions[currency].params[1] - 1) * factor[[currency]].dropna()

pd.DataFrame(index=['Count - Daily Premium > 0', 'Count - Days', 'Frequency (%) - Positive Premium'],
             columns=pred_xs_log_rets.columns,
             data = [(pred_xs_log_rets > 0).sum().map("{:.0f}".format), 
                     pred_xs_log_rets.count().map("{:.0f}".format), 
                     (pred_xs_log_rets > 0).mean().map("{:.2%}".format)]).T